# Cleanup SageMaker Resources

Delete SageMaker endpoint resources by providing the **model name**.

Deletion order: Inference Component → Endpoint → Endpoint Config → Model

In [ ]:
import boto3
import time

# Configuration
region = "eu-west-2"
model_name = "CHANGE"  # <-- CHANGE THIS

# Derived names (matches 03_deploy_and_evaluate.ipynb convention)
endpoint_config_name = f"epc-{model_name}"
endpoint_name = f"ep-{model_name}"
inference_component_name = f"ic-{model_name}"

print(f"Will delete:")
print(f"  Inference Component: {inference_component_name}")
print(f"  Endpoint:            {endpoint_name}")
print(f"  Endpoint Config:     {endpoint_config_name}")
print(f"  Model:               {model_name}")

In [ ]:
sm_client = boto3.client("sagemaker", region_name=region)

def delete_resource(delete_fn, name, rtype):
    try:
        delete_fn()
        print(f"✓ Deleted {rtype}: {name}")
    except sm_client.exceptions.ResourceNotFound:
        print(f"⊘ {rtype} not found: {name}")
    except Exception as e:
        print(f"✗ Failed to delete {rtype} {name}: {e}")

# 1. Delete inference component
delete_resource(
    lambda: sm_client.delete_inference_component(InferenceComponentName=inference_component_name),
    inference_component_name, "Inference component"
)

print("Waiting 10s for inference component deletion...")
time.sleep(10)

# 2. Delete endpoint
delete_resource(
    lambda: sm_client.delete_endpoint(EndpointName=endpoint_name),
    endpoint_name, "Endpoint"
)

# 3. Delete endpoint config
delete_resource(
    lambda: sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name),
    endpoint_config_name, "Endpoint config"
)

# 4. Delete model
delete_resource(
    lambda: sm_client.delete_model(ModelName=model_name),
    model_name, "Model"
)

print("\n✓ Cleanup complete!")